In [6]:
from collections import deque
import numpy as np

In [7]:
class Problem:
    '''
    Abstract base class for problem formulation.
    It declares the expected methods to be used by a search algorithm.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''

    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the initial state and, if applicable, the goal state.'''
        self.init_state = None

    def actions(self, state):
        '''Returns an iterable with the applicable actions to the given state.'''
        raise NotImplementedError

    def result(self, state, action):
        '''Returns the resulting state from applying the given action to the given state.'''
        raise NotImplementedError

    def goal_test(self, state):
        '''Returns whether or not the given state is a goal state.'''
        raise NotImplementedError

    def step_cost(self, state, action):
        '''Returns the step cost of applying the given action to the given state.'''
        raise NotImplementedError


class Node:
    '''Node data structure for search space bookkeeping.'''

    def __init__(self, state, parent, action, path_cost):
        '''Constructor for the node state with the required parameters.'''
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    @classmethod
    def root(cls, init_state):
        '''Factory method to create the root node.'''
        return cls(init_state, None, None, 0)

    @classmethod
    def child(cls, problem, parent, action):
        '''Factory method to create a child node.'''
        return cls(
            problem.result(parent.state, action),
            parent,
            action,
            parent.path_cost + problem.step_cost(parent.state, action))
    
def solution(node):
    '''A method to extract the sequence of actions representing the solution from the goal node.'''
    actions = []
    cost = node.path_cost
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost


In [13]:
class Node:
    '''Node data structure for search space bookkeeping.'''

    def __init__(self, state, parent, action, path_cost):
        '''Constructor for the node state with the required parameters.'''
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    @classmethod
    def root(cls, init_state):
        '''Factory method to create the root node.'''
        return cls(init_state, None, None, 0)

    @classmethod
    def child(cls, problem, parent, action):
        '''Factory method to create a child node.'''
        return cls(
            problem.result(parent.state, action),
            parent,
            action,
            parent.path_cost + problem.step_cost(parent.state, action))
def bfs_graph(problem, verbose=False):
    '''Breadth-first graph search implementation.'''
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    frontier = deque([problem.init_state])
    explored = {problem.init_state}
    path = []
    #if verbose: visualizer = Visualizer(problem)
    while frontier:
     #   if verbose: visualizer.visualize(frontier)
        state = frontier.pop()
        for action in problem.actions(state):
            child_state =  problem.result(state, action)
            path.append(action)
            if child_state not in explored:
                if problem.goal_test(child_state):
                    return path
                frontier.appendleft(child_state)
                explored.add(child_state)

In [3]:
def bfs_tree(problem, verbose=False):
    '''Breadth-first tree search implementation.'''
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    frontier = deque([Node.root(problem.init_state)])
    #print(problem.init_state)
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.pop()
        print(type(node.state))
        for action in problem.actions( node.state):
            child = Node.child(problem, node, action)
            if problem.goal_test(child.state):
                return solution(child)
            frontier.appendleft(child)

def bfs_graph(problem, verbose=False):
    '''Breadth-first graph search implementation.'''
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    frontier = deque([Node.root(problem.init_state)])
    explored = {problem.init_state}
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.pop()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                if problem.goal_test(child.state):
                    return solution(child)
                frontier.appendleft(child)
                explored.add(child.state)



In [4]:
#################### PART A #############################

In [11]:
def dfs_tree(problem, verbose=False):
    '''Depth-first tree search implementation.'''
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    frontier = deque([Node.root(problem.init_state)])
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.pop()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if problem.goal_test(child.state):
                return solution(child)
            frontier.append(child)

def dfs_graph(problem, verbose=False):
    '''Depth-first graph search implementation.'''
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    frontier = deque([Node.root(problem.init_state)])
    explored = {problem.init_state}
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.pop()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                if problem.goal_test(child.state):
                    return solution(child)
                frontier.append(child_state)
                explored.add(child_state)

In [ ]:
############################ PART B ###################################

In [9]:
# formation of the problem
class Maze(Problem):
    '''3x3 Sliding Puzzle problem formulation.'''

    def __init__(self, x1, init_state, goal_state):
        # assert init_state.count('*') == (1,1)
        # assert goal_state.count('*') == (1,1)
        self.init_state = init_state
        self._goal_state = goal_state
        self._m = x1
        self._action_values = {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}

    def actions(self, state):
        # up
        if state[0] - 1 > 0:
            if self._m[(state[0] - 1, state[1])] != '#':
                # update # swap
                yield 'up'
        # down
        if state[0] + 1 < 5:
            if self._m[(state[0] + 1, state[1])] != '#':
                yield 'down'
        # left
        if state[1] - 1 > 0:
            if self._m[(state[0], state[1] - 1)] != '#':
                yield 'left'
        # right
        if state[0] - 1 < 21:
            if self._m[(state[0], state[1] + 1)] != '#':
                yield 'right'

    def result(self, state, action):
        def swap(i, j):
            '''Auxiliary function for swapping two elements in a tuple.'''
            self._m[i], self._m[j] = self._m[j], self._m[i]
            state = j
            return state

        return swap(state, (state[0] + self._action_values[action][0], state[1] + self._action_values[action][1]))

    def goal_test(self, state):
        return state == self._goal_state

    def step_cost(self, state, action):
        return 1

In [14]:
# processing for the problem

maze =  '''
   #                #
   #                #
          ####      #
  ##      #     #####
  *#      #      +   
'''

maze = maze.replace(' ','0')
mazee = list(maze)
print(mazee)
print(len(mazee))
mazee.remove('\n')
print(len(mazee))
x = np.reshape( mazee , (5,22) )
x = x [x != '\n']
x1 = np.reshape( x , (5,21) )

z = np.where(x1 == '+')
z1  = list(z[0])[0]
z2 =  list(z[1])[0]
goal= (z1 ,z2)

z = np.where(x1 == '*')
z1  = list(z[0])[0]
z2 =  list(z[1])[0]
init= (z1 ,z2)

problem = Maze( x1 , init , goal)
bfs_graph(problem)

['\n', '0', '0', '0', '#', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '#', '\n', '0', '0', '0', '#', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '#', '\n', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '#', '#', '#', '#', '0', '0', '0', '0', '0', '0', '#', '\n', '0', '0', '#', '#', '0', '0', '0', '0', '0', '0', '#', '0', '0', '0', '0', '0', '#', '#', '#', '#', '#', '\n', '0', '0', '*', '#', '0', '0', '0', '0', '0', '0', '#', '0', '0', '0', '0', '0', '0', '+', '0', '0', '0', '\n']
111
110


['left',
 'up',
 'right',
 'up',
 'down',
 'up',
 'down',
 'right',
 'down',
 'right',
 'up',
 'left',
 'right',
 'down',
 'left',
 'left',
 'right',
 'up',
 'down',
 'left',
 'right',
 'down',
 'right',
 'up',
 'down',
 'right',
 'up',
 'down',
 'left',
 'right',
 'down',
 'left',
 'right',
 'up',
 'right',
 'up',
 'down',
 'left',
 'right',
 'up',
 'down',
 'left',
 'right',
 'down',
 'left',
 'right',
 'up',
 'left',
 'right',
 'up',
 'down',
 'left',
 'right',
 'up',
 'down',
 'left',
 'right',
 'down',
 'left',
 'right',
 'up',
 'left',
 'right',
 'up',
 'down',
 'left',
 'right',
 'up',
 'down',
 'left',
 'right',
 'down',
 'left',
 'right',
 'up',
 'left',
 'right',
 'up',
 'down',
 'left',
 'right',
 'up',
 'down',
 'left',
 'down',
 'left',
 'right',
 'up',
 'left',
 'right',
 'up',
 'down',
 'left',
 'left',
 'right',
 'up',
 'left',
 'left',
 'right',
 'left',
 'right',
 'left',
 'right',
 'down',
 'left',
 'right',
 'up',
 'down',
 'right',
 'down',
 'left',
 'right',
 'up'